# Trabalho 1 - EEL891

Etapas que serão realizadas a seguir:
- Implementar o pré-processamento
- Visualizar, analisar e selecionar os atributos
- Escolher os métodos que serão usados e compará-los
- Verificar o desempenho com validação cruzada
- Ajustas os hiperparâmetros para chegar ao melhor resultado possível
- Treinar o modelo com o conjunto de teste

### Importação das bibliotecas

In [1]:
import pandas as pd
import numpy  as np

from matplotlib                 import pyplot as plt

from sklearn.naive_bayes        import MultinomialNB, GaussianNB
from sklearn.model_selection    import cross_val_predict, train_test_split
from sklearn.metrics            import accuracy_score
from sklearn.neighbors          import KNeighborsClassifier
from sklearn.preprocessing      import LabelBinarizer, MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.linear_model       import LogisticRegression
from scipy.stats                import pearsonr

### Pré-processamento

Agora, o próximo passo é importar o conjunto de treinamento:

In [2]:
dados = pd.read_csv(r"C:\Users\andre\Documents\UFRJ\SÉTIMO PERÍODO\Introdução ao Aprendizado de Máquina\EEL891-Trabalho-1\conjunto_de_treinamento.csv")
dados = dados.iloc[:,:]

Embaralhamento dos dados e separá-los em atributos e alvo (convertendo para um array):

In [3]:
dados = dados.sample(frac=1)
x = dados.iloc[:,:-1].to_numpy()
y = dados.iloc[:,-1].to_numpy()

Descartando algumas informações que não são relevantes. O campo "id_solicitante" é um número de 1 a 20000, ou seja, apenas atrapalharia o modelo. Os campos "grau_instrucao" e "possui_telefone_celular" não foram extraídos corretamente, então segui a orientação do professor para descartá-los. E o campo "qtde_contas_bancarias_especiais" é exatamente igual ao campo "qtde_contas_bancarias", logo foi descartado seguindo a orientação do professor.

In [4]:
dados = dados.drop(['id_solicitante', 'grau_instrucao', 'possui_telefone_celular', 'qtde_contas_bancarias_especiais'],axis=1)

Identificação das variáveis categóricas olhando o dicionário de dados:

In [5]:
variaveisCategoricas = [x for x in dados.columns if dados[x].dtype=='object' or x == 'produto_solicitado' or x == 'tipo_endereco' or x =='estado_civil' or  x =='nacionalidade' or x=='tipo_residencia' or x=='possui_email' or x=='possui_cartao_visa' or x=='possui_cartao_mastercard' or x=='possui_cartao_diners' or x=='possui_cartao_amex' or x=='possui_cartao_cartoes' or x=='possui_carro' or x=='profissao' or x=='ocupacao' or x=='profissao_companheiro' or x=='local_onde_reside' or x=='local_onde_trabalha']
print(variaveisCategoricas)

['produto_solicitado', 'forma_envio_solicitacao', 'tipo_endereco', 'sexo', 'estado_civil', 'nacionalidade', 'estado_onde_nasceu', 'estado_onde_reside', 'possui_telefone_residencial', 'codigo_area_telefone_residencial', 'tipo_residencia', 'possui_email', 'possui_cartao_visa', 'possui_cartao_mastercard', 'possui_cartao_diners', 'possui_cartao_amex', 'possui_carro', 'vinculo_formal_com_empresa', 'estado_onde_trabalha', 'possui_telefone_trabalho', 'codigo_area_telefone_trabalho', 'profissao', 'ocupacao', 'profissao_companheiro', 'local_onde_reside', 'local_onde_trabalha']


Verificando as variáveis categóricas (quantidade de categorias e quais são):

In [6]:
for v in variaveisCategoricas:
    print("\n%15s: " %v, "%4d categorias" % len(dados[v].unique() ) )
    print(dados[v].unique(), "\n")


produto_solicitado:     3 categorias
[1 2 7] 


forma_envio_solicitacao:     3 categorias
['internet' 'presencial' 'correio'] 


  tipo_endereco:     2 categorias
[1 2] 


           sexo:     4 categorias
['M' 'F' 'N' ' '] 


   estado_civil:     8 categorias
[2 3 4 6 1 5 7 0] 


  nacionalidade:     3 categorias
[1 0 2] 


estado_onde_nasceu:    28 categorias
['MG' 'RJ' 'AC' 'BA' ' ' 'SC' 'RN' 'SP' 'CE' 'PA' 'AP' 'RS' 'PE' 'MA' 'AL'
 'GO' 'PI' 'PR' 'DF' 'PB' 'TO' 'MS' 'MT' 'ES' 'AM' 'RO' 'SE' 'RR'] 


estado_onde_reside:    27 categorias
['MG' 'SP' 'BA' 'TO' 'SC' 'RN' 'PA' 'AP' 'CE' 'RS' 'PE' 'RJ' 'AL' 'PI'
 'PB' 'MS' 'GO' 'DF' 'MA' 'PR' 'ES' 'MT' 'AM' 'SE' 'AC' 'RO' 'RR'] 


possui_telefone_residencial:     2 categorias
['N' 'Y'] 


codigo_area_telefone_residencial:    81 categorias
[' ' '5' '84' '50' '105' '117' '123' '54' '39' '58' '97' '22' '100' '68'
 '15' '16' '110' '33' '103' '120' '12' '69' '81' '66' '20' '32' '29' '14'
 '18' '107' '40' '67' '38' '75' '25' '119' '125' '44' '

Correção de alguns valores (principalmente em branco). A ideia inicial é criar uma nova categoria para os valores em branco, e depois tentar excluí-los para ver qual é a resposta dos modelos:

In [7]:
for x in variaveisCategoricas:
    if x == 'sexo':
        dados[x].replace(' ', 'N', inplace=True)
    if x == 'estado_onde_nasceu':
        dados[x].replace(' ', 'NN', inplace=True)
    if x == 'codigo_area_telefone_residencial':
        dados[x].replace(' ', '000', inplace=True)
    if x == 'tipo_residencia':
        dados[x].replace(0, 6, inplace=True)
        dados[x].fillna(0, inplace=True)
    if x == 'estado_onde_trabalha':
        dados[x].replace(' ', 'NN', inplace=True)
    if x == 'codigo_area_telefone_trabalho':
        dados[x].replace(' ', '000', inplace=True)
    if x == 'profissao':
        dados[x].replace(0, 18, inplace=True)
        dados[x].fillna(0, inplace=True)
    if x == 'ocupacao':
        dados[x].replace(0, 6, inplace=True)
        dados[x].fillna(0, inplace=True)
    if x == 'profissao_companheiro':
        dados[x].replace(0, 18, inplace=True)
        dados[x].fillna(0, inplace=True)
    print("\n%15s: " %x, "%4d categorias" % len(dados[x].unique() ) )
    print(dados[x].unique(), "\n")


produto_solicitado:     3 categorias
[1 2 7] 


forma_envio_solicitacao:     3 categorias
['internet' 'presencial' 'correio'] 


  tipo_endereco:     2 categorias
[1 2] 


           sexo:     3 categorias
['M' 'F' 'N'] 


   estado_civil:     8 categorias
[2 3 4 6 1 5 7 0] 


  nacionalidade:     3 categorias
[1 0 2] 


estado_onde_nasceu:    28 categorias
['MG' 'RJ' 'AC' 'BA' 'NN' 'SC' 'RN' 'SP' 'CE' 'PA' 'AP' 'RS' 'PE' 'MA'
 'AL' 'GO' 'PI' 'PR' 'DF' 'PB' 'TO' 'MS' 'MT' 'ES' 'AM' 'RO' 'SE' 'RR'] 


estado_onde_reside:    27 categorias
['MG' 'SP' 'BA' 'TO' 'SC' 'RN' 'PA' 'AP' 'CE' 'RS' 'PE' 'RJ' 'AL' 'PI'
 'PB' 'MS' 'GO' 'DF' 'MA' 'PR' 'ES' 'MT' 'AM' 'SE' 'AC' 'RO' 'RR'] 


possui_telefone_residencial:     2 categorias
['N' 'Y'] 


codigo_area_telefone_residencial:    81 categorias
['000' '5' '84' '50' '105' '117' '123' '54' '39' '58' '97' '22' '100' '68'
 '15' '16' '110' '33' '103' '120' '12' '69' '81' '66' '20' '32' '29' '14'
 '18' '107' '40' '67' '38' '75' '25' '119' '125' '44' '4

In [8]:
for x in dados:
    if x == 'meses_na_residencia':
        dados[x].fillna(300, inplace=True)
    if x == 'grau_instrucao_companheiro':
        dados[x].fillna(6, inplace=True)
    print("\n%15s: " %x, "%4d categorias" % len(dados[x].unique() ) )
    print(dados[x].unique(), "\n")


produto_solicitado:     3 categorias
[1 2 7] 


 dia_vencimento:     6 categorias
[10 15  5  1 20 25] 


forma_envio_solicitacao:     3 categorias
['internet' 'presencial' 'correio'] 


  tipo_endereco:     2 categorias
[1 2] 


           sexo:     3 categorias
['M' 'F' 'N'] 


          idade:    84 categorias
[ 47  70  54  46  28  85  50  33  57  53  30  52  32  65  26  38  48  42
  31  49  24  40  58  41  74  21  44  39  51  78  36  60  19  43  20  18
  61  25  29  37  72  27  81  83  64  55  68  45  35  56  59  23  73  67
  34  69  86  22  62  71  63  75  66  89  79  77  95  80  82  76  17  88
  87  84  90  94  93  91  96 100 106  97  92   7] 


   estado_civil:     8 categorias
[2 3 4 6 1 5 7 0] 


qtde_dependentes:    15 categorias
[ 2  0  4  1  3  6  5 11  8  7 10  9 14 13 53] 


  nacionalidade:     3 categorias
[1 0 2] 


estado_onde_nasceu:    28 categorias
['MG' 'RJ' 'AC' 'BA' 'NN' 'SC' 'RN' 'SP' 'CE' 'PA' 'AP' 'RS' 'PE' 'MA'
 'AL' 'GO' 'PI' 'PR' 'DF' 'PB' 'TO' 'MS' 'MT' '

Binarizar e fazer o one hot encoding de alguns parâmetros:

In [9]:
binarizador = LabelBinarizer()
for x in ['tipo_endereco',
          'possui_telefone_residencial',
          'vinculo_formal_com_empresa',
          'possui_telefone_trabalho']:
    dados[x] = binarizador.fit_transform(dados[x])
#print(dados.head(4).T)


dados = pd.get_dummies(dados, columns=['forma_envio_solicitacao',
                                       'sexo'
                                       #'estado_onde_nasceu',
                                       #'estado_onde_reside',
                                       #'estado_onde_trabalha'
                                      ])
print(dados.head(4).T)

                                     172    12605  13533  13496
produto_solicitado                       1      1      2      1
dia_vencimento                          10     15     10     10
tipo_endereco                            0      0      0      0
idade                                   47     70     54     46
estado_civil                             2      2      3      2
qtde_dependentes                         2      0      0      0
nacionalidade                            1      1      1      1
estado_onde_nasceu                      MG     RJ     MG     AC
estado_onde_reside                      MG     SP     MG     SP
possui_telefone_residencial              0      1      0      1
codigo_area_telefone_residencial       000      5    000      5
tipo_residencia                        1.0    6.0    1.0    2.0
meses_na_residencia                    9.0    0.0    3.0    0.0
possui_email                             1      0      1      0
renda_mensal_regular                2353

Verificar a distribuição entre os possíveis valores dentro dos parâmetros:

In [10]:
for x in dados:
    print(dados[x].value_counts())

produto_solicitado
1    17023
2     2435
7      542
Name: count, dtype: int64
dia_vencimento
10    7847
15    3557
25    3089
5     2825
20    1952
1      730
Name: count, dtype: int64
tipo_endereco
0    19873
1      127
Name: count, dtype: int64
idade
40     555
39     534
36     526
32     518
37     513
      ... 
94       2
96       2
100      1
97       1
7        1
Name: count, Length: 84, dtype: int64
estado_civil
2    10088
1     6519
4     1573
6      763
5      522
3      234
7      220
0       81
Name: count, dtype: int64
qtde_dependentes
0     13350
1      2814
2      2189
3      1029
4       352
5       149
6        57
7        22
8        14
9         9
10        7
11        4
13        2
14        1
53        1
Name: count, dtype: int64
nacionalidade
1    19152
0      808
2       40
Name: count, dtype: int64
estado_onde_nasceu
BA    2351
SP    2336
RS    1919
CE    1910
PE    1651
MG    1446
RN     827
NN     822
PR     764
RJ     720
AL     678
PA     676
PB     608
MA 

Verificar a correlação de pearson entre os parâmetros e o alvo (obs: os parâmetros não parecem muito promissores, mas veremos mais na hora de treinar o modelo):

In [11]:
colunas = dados.columns
for col in colunas:
    if type(dados[col][0]) != str:
        corr = pearsonr(dados[col], dados['inadimplente'])
        print(col, " = %.4f" % corr[0], " , %.10f" % corr[1])

produto_solicitado  = 0.0301  , 0.0000206384
dia_vencimento  = 0.0803  , 0.0000000000
tipo_endereco  = -0.0044  , 0.5332219466
idade  = -0.1207  , 0.0000000000
estado_civil  = -0.0313  , 0.0000097351
qtde_dependentes  = 0.0176  , 0.0126681749
nacionalidade  = 0.0005  , 0.9442714564
possui_telefone_residencial  = -0.0808  , 0.0000000000
tipo_residencia  = 0.0200  , 0.0046491853
meses_na_residencia  = -0.0047  , 0.5024577844
possui_email  = -0.0057  , 0.4168413604
renda_mensal_regular  = -0.0009  , 0.8958507584
renda_extra  = 0.0059  , 0.4079845863
possui_cartao_visa  = -0.0013  , 0.8559114105
possui_cartao_mastercard  = -0.0200  , 0.0047680534
possui_cartao_diners  = 0.0025  , 0.7234775228
possui_cartao_amex  = -0.0011  , 0.8757773534
possui_outros_cartoes  = -0.0032  , 0.6543764344
qtde_contas_bancarias  = 0.0141  , 0.0456463167
valor_patrimonio_pessoal  = 0.0012  , 0.8627919655
possui_carro  = 0.0128  , 0.0707060897
vinculo_formal_com_empresa  = 0.0062  , 0.3773057238
possui_telefone_

In [13]:
#atributos usados pelo classificador
atributosSelecionados = [
    'produto_solicitado',
    'dia_vencimento',
    'forma_envio_solicitacao_correio',
    'forma_envio_solicitacao_internet',
    'forma_envio_solicitacao_presencial',
    'tipo_endereco',
    'sexo_M',
    'sexo_F',
    'sexo_N',
    'idade',
    'estado_civil',
    'qtde_dependentes',
    'nacionalidade',
    #'estado_onde_nasceu',
    #'estado_onde_reside',
    'possui_telefone_residencial',
    #'codigo_area_telefone_residencial',
    'tipo_residencia',
    'meses_na_residencia',
    'possui_email',
    'renda_mensal_regular',
    'renda_extra',
    'possui_cartao_visa',
    'possui_cartao_mastercard',
    'possui_cartao_diners',
    'possui_cartao_amex',
    'possui_outros_cartoes',
    'qtde_contas_bancarias',
    'valor_patrimonio_pessoal',
    'possui_carro',
    'vinculo_formal_com_empresa',
    #'estado_onde_trabalha',
    'possui_telefone_trabalho',
    #'codigo_area_telefone_trabalho',
    'meses_no_trabalho',
    'profissao',
    'ocupacao',
    'profissao_companheiro',
    'grau_instrucao_companheiro',
    'local_onde_reside',
    'local_onde_trabalha',
    'inadimplente'
]

dados = dados[atributosSelecionados]

#embaralhar o conjunto de dados
dadosEmbaralhados = dados.sample(frac=1, random_state=1)

#criar arrays x e y separando os parâmetros do alvo
x = dadosEmbaralhados.iloc[:,:-1].to_numpy()
y = dadosEmbaralhados.iloc[:,-1].to_numpy()

#separar em conjunto de treino e de teste
amostrasTreinamento = 15000

xTreino = x[:amostrasTreinamento,:]
yTreino = x[:amostrasTreinamento]

xTeste = x[amostrasTreinamento:,:]
yTeste = y[amostrasTreinamento:]

#ajustar a escala
scaler = MinMaxScaler()
scaler.fit(xTreino)

xTreino = scaler.transform(xTreino)
xTeste = scaler.transform(xTeste)

#treinar classificador KNN
classificador = KNeighborsClassifier(n_neighbors=5)

classificador = classificador.fit(xTreino, yTreino)

#obter as respostas do classificador
yRespostaTreino = classificador.predict(xTreino)
yRespostaTeste = classificador.predict(xTeste)

#desempenho na amostra de treino
total = len(yTreino)
acertos = sum(yRespostaTreino==yTreino)
erros = sum(yRespostaTreino!=yTreino)

print("Total de amostras: ", total)
print("Respostas corretas: ", acertos)
print("Respostas erradas: ", erros)

acuracia = acertos / total

print("Acurácia: %.1f %%" % (100*acuracia) )

#verificar a variacao da acuracia com o numero de vizinhos

#verificar a variacao da acuracia para regressao logistica

[1 0 0 ... 1 0 1]


ValueError: Unknown label type: unknown. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.

### Treinamento do modelo

Classificador Bayesiano com Distribuição Multinomial:

In [ ]:
classificador = MultinomialNB(alpha=1.0)
y_pred = cross_val_predict(classificador, x, y, cv=5)

print("Acurácia = %6.4f" % accuracy_score(y,y_pred))

Classificador Bayesiano com Distribuição Multinomial:

In [ ]:
classificador = GaussianNB(alpha=1.0)
y_pred = cross_val_predict(classificador, x, y, cv=5)

print("Acurácia = %6.4f" % accuracy_score(y,y_pred))

Classificador KNN

In [ ]:
print("\n  K  TREINO  TESTE")
print(" --  ------  ------")

for k in range(1,50):
    classificador = KNeighborsClassifier(
        n_neighbors=k,
        weights = 'uniform',
        p=1
    )
    classificador = classificador.fit(xTreino, yTreino)
    
    yRespostaTreino = classificador.predict(xTreino)
    yRespostaTeste = classificador.predict(xTeste
    
    acuraciaTreino = sum(yRespostaTreino==yTreino)/len(yTreino)
    acuraciaTeste = sum(yRespostaTeste==yTeste)/len(yTeste)
    
    print(
        "%3d" % k,
        "%6.1f" % (100*acuraciaTreino)
        "%6.1f" % (100*acuraciaTeste)
    )
    


Classificador de Regressão Logística

In [ ]:
print("\n         C  TREINO  TESTE")
print(" ----------  ------  ------")

for k in range(-6,7):
    c = pow(10,k)
    classificador = LogisticRegression(
        penalty = 'l2',
        C = c
    )
    classificador = classificador.fit(xTreino, yTreino)
    
    yRespostaTreino = classificador.predict(xTreino)
    yRespostaTeste = classificador.predict(xTeste)
    
    acuraciaTreino = sum(yRespostaTreino==yTreino)/len(yTreino)
    acuraciaTeste = sum(yRespostaTeste==yTeste)/len(yTeste)
    
    print(
        "%14.6f" % c,
        "%6.1f" % (100*acuraciaTreino)
        "%6.1f" % (100*acuraciaTeste)
    )